In [1]:
from ase.io import read
from collections import defaultdict
import numpy as np

In [2]:
#Read in xyz trajectory
traj = read("test-traj.xyz", format="xyz", index=':') #https://wiki.fysik.dtu.dk/ase/ase/io/io.html

#Also be able to ready in PDB / GRO / ?

In [ ]:
#Trajectories will be read in as a list of ASE Atoms units
traj
#Single structs will be a single ASE Atoms unit -- differentiate between them?

In [5]:
# Get central ion ID
# User can provide:
#                 Element
#                 Atom Index
# Can also be MORE THAN ONE Element/Atom Index
metal_ids, element_ids = [], []
metals = ["Eu"]
elements = ['O','N']


for i,x in enumerate(traj[0]):
    if x.symbol in metals: 
        metal_ids.append(i)
    elif x.symbol in elements: 
        element_ids.append(i)
print(metal_ids, element_ids)

[0] [1, 2, 13, 14, 15, 16, 17, 18, 19, 20, 33, 36, 39]


In [46]:
#Calculate RDF between ions and elements of interest
traj_nearby_ids_set = defaultdict(set)
rdf_frames = []
rdf_int_frames = []
rdf_max = 6.0 # Angstroms
shell_max = 5.0 # Angstroms - to help shorten first-shell geometry analyses, store only atom IDs that ever occur within 5A of metal
bin_width = 0.02
bin_range = np.arange(0, rdf_max+bin_width, bin_width)

for frame in traj:
    for metal_id in metal_ids:

        f_dists = frame.get_distances(metal_id, element_ids)

        ### To save time later, keep indices of atoms that enter first shell over traj
        near_ids_frame = [element_ids[x] for x in (f_dists<shell_max).nonzero()[0]]
        traj_nearby_ids_set[metal_id] |= set(near_ids_frame)

        bin_counts = np.bincount(np.searchsorted(bin_range, f_dists, side="left"))  # this bins the distances to bin_range 
                                                                                    # bin_counts[0] will be dists less than 0 (presumably none)
                                                                                    # bin_counts[-1] will either be the number of dists > rdf_max (if there are any) or 
                                                                                    #                                   count of dists <= a value less than rdf_max (will need padding)
        if len(bin_counts) == len(bin_range)+1: #case where there are dists > rdf_max
            bins = bin_counts[1:-1] # ignore bin_counts[0] and ignore counts of elements greater than rdf_max
            rdf_int_frames.append(np.cumsum(bins).copy())
        else:
            bins = bin_counts[1:] # ignore bin_counts[0] -- will need to pad with zeros to complete bin range
            rdf_int_frames.append(np.pad(np.cumsum(bins), (0, len(bin_range)-len(bins)), "edge").copy())
        
        dists_count = sum(bins)
        density_A = 1.0 / (4.0/3.0 * np.pi * np.power(rdf_max, 3))
        bins = 1 / (density_A*dists_count)*bins / (4.0*np.pi*np.arange(1,len(bins)+1,1)**2 * bin_width**3)

        if len(bins) < len(bin_range):
            rdf_frames.append(np.pad(bins, (0, len(bin_range)-len(bins)), "constant").copy())
        else:
            rdf_frames.append(bins.copy())

rdf = np.mean(rdf_frames, axis=0) #Calculate RDF of trajectory
rdf_int = np.mean(rdf_int_frames, axis=0) #Calculate integral of RDF of trajectory

#print(traj_nearby_ids_set)

#for i,j in enumerate(bin_range):
#    print(f'{j}\t{rdf[i]}\t{rdf_int[i]}')


defaultdict(<class 'set'>, {0: {1, 2, 13, 14, 15, 16, 17, 18, 19, 20, 33, 36, 39}})
